In [1]:
import sys
print(sys.path)


import torch
import cv2
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import numpy as np
import os


['/Users/sudhanshu/anaconda3/lib/python311.zip', '/Users/sudhanshu/anaconda3/lib/python3.11', '/Users/sudhanshu/anaconda3/lib/python3.11/lib-dynload', '', '/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/vqa/lib/python3.11/site-packages']


In [2]:
from models import NERModel, SamModelPrediction,NERRModel
from segmentation import create_segmented_image
import json
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/vqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


final text_encoder_type: bert-base-uncased
Model loaded from /Users/sudhanshu/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [3]:
ner = NERRModel("../lang-segment-anything/fineTunedT5ForNER")
#ner=NERModel()
sam_predictor = SamModelPrediction()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


final text_encoder_type: bert-base-uncased
Model loaded from /Users/sudhanshu/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [4]:
# Function to extract entities using Transformer model
# def extract_entities(question):
#     results = ner.extract_entities(question)
#     return [(res['word'], res['entity']) for res in results]


def extract_entities(sentence):
    instruction = f" Extract exact words from the sentence that are place, animal, thing or location"
    
    prompt = f"""
    1. Sentence: "The cat sat on the mat?" Common Nouns: cat, sat, mat.
    2. Sentence: "He drinks a lot of coffee and reads many books." Common Nouns:  coffee, reads, books.
    3. Sentence: "Sky is blue and grass is green." Common Nouns: sky, blue, grass, green.
    4. Sentence: "Is the  Boy is skateboarding on the wall?" Common Nouns: boy, skateboarding, wall .
    5. Sentence: "What color is teddy bear " Common Nouns: teddy bear, color
    6. Sentence: "Teddy bear is pink ?" Common Nouns: teddy bear, pink
    Sentence: "{sentence}"
    Common Nouns:"""
    # Encode the prompt
    
    
    # Generate output
    outputs = ner.extract_entities(instruction,prompt)

    if ',' in outputs:
        # Split the string at each comma and strip whitespace
        entity_list = [entity.strip() for entity in outputs.split(',')]
    else:
        # Return the entire string as a single element list, also strip any whitespace
        entity_list = [outputs.strip()]
    return entity_list

 # Function to display image and entities

def display_image_with_sam(image_id, entities, model):
    base_url = "http://images.cocodataset.org/val2014"
    image_url = f"{base_url}/COCO_val2014_{image_id:012d}.jpg"
    
    # Fetch the image
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    
    # Use SAM model to get segmentation masks for each entity
    #prompts = [entity[0] for entity in entities]  # Assuming you want to segment based on the full entity text
   
    masks = [model.predict_image(image, ent) for ent in entities]
    
    masks = [item for sublist in masks for item in sublist]
   
    segmentedImageWithEntities = create_segmented_image(image,masks)
  
    # Display the original image and masks


    return segmentedImageWithEntities

def create_unique_filename(image_id):
    # Create a unique filename using the image ID and the current timestamp
   
    return f"{image_id}_image.png"

In [5]:
def batch_extract_entities(questions):
    # Modify this function to handle batch processing.
    # This is a placeholder. You'll need to implement batch processing in your actual entity extraction model.
    return [extract_entities(question) for question in questions]

def process_image(data):
    try:
        question, image_id, question_id, entities = data
        directory = "src/segmentedImage"
        name = f"{image_id}_{question_id}"
        unique_filename = create_unique_filename(name)
        segmented_image = display_image_with_sam(image_id, entities, sam_predictor)
        segmented_image = segmented_image.convert('RGB')
        segmented_image.save(os.path.join(directory, unique_filename))
        return unique_filename, question, question_id
    except Exception as e:
        print(f"Error processing image {image_id}: {str(e)}")
        return None

In [8]:
from concurrent.futures import ProcessPoolExecutor
file_path = '../dataset/combined_data.json'
with open(file_path, 'r') as file:
    data = json.load(file)
directory = "../segmentedImage"
# Extract questions and image IDs
questions_and_ids_question_id = [(item['question'], item['image_id'],item['question_id']) for item in data['annotations']]

# # Load Transformer NER model
questionsSegmentedImageMapping ={}
questions_idSegmentedImageMapping ={}

# all_questions = [item[0] for item in questions_and_ids_question_id]
# entities= batch_extract_entities(all_questions)
# data_to_process = [(q, iid, qid, entities) for (q, iid, qid), entities in zip(questions_and_ids_question_id, entities)]


# Store results in dictionaries


for question, image_id,question_id in questions_and_ids_question_id:
    # Extract entities from the question
    entities = extract_entities(question)
    print(entities)
    entities =  [item[0] for item in entities]
    #entities=["people","background"]
    # for entity in entities:
       
    #     if question.lower() in entity.lower():
           
          
    #         entities=[]
    
    
   
  
   
    
    # Generate a segmented image based on the image ID and entities
    # This function should handle the creation and saving of the image

    name= str(image_id)+"_"+ str(question_id)
    unique_filename = create_unique_filename(name)
    segmented_image = display_image_with_sam(image_id, entities, sam_predictor)
    segmented_image.convert('RGB')
    segmented_image.save(os.path.join(directory, unique_filename))
    # # Save the segmented image with a unique filename

    
    # # Map the unique image filename to the question
    questionsSegmentedImageMapping[unique_filename] = question
    questions_idSegmentedImageMapping[unique_filename] =question_id

print("done")

['where is he looking?']


['background', 'people']
['he']
['website']
['soup']
['rice noodle soup']
['right', 'soup']
['street', 'thing']
['photos']
['truck', 'sell']
done


In [ ]:
question= "What are background people doing ?"
ex= extract_entities(question)
print(ex)

['background people']


In [9]:
with open('../dataset/question_image_mapping.json', 'w') as f:
    json.dump(questionsSegmentedImageMapping, f)

with open('../dataset/questionId_image_mapping.json', 'w') as f:
    json.dump(questions_idSegmentedImageMapping, f)